# Import and High-Level Setup

In [1]:
# General Python Packages
import os
import time

# Torch Packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import lr_scheduler, SGD
from torch.autograd import Variable
from torch import nn
from torch.nn import DataParallel
from torch.nn import Module

# General Analytics Packages
import pandas as pd
import numpy as np

# Visualization / Image Packages
import matplotlib.pyplot as plt
from PIL import Image

# Randomization Functions
from random import random as randuni

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
# Put MatPlotLib in interactive mode
plt.ion()

# Define Data Manipulation Classes

### Helper Utility Classes

In [3]:
def is_image_file(fname):
    """Checks if a file is an image.
    Args:
        fname (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return fname.lower().endswith('.png')

def create_label_maps(details_df):
    """ Take a descriptive dataframe and extract the unique labels and map to index values
    Args:
        details_df: Dataframe with the image details
    Returns:
        label_list: list of unique labels in the dataframe
        label_to_index: map from labels to indices
    """
    """ TODO: Research paper also excludes these labels but need to figure out how to handle
              cases that have these as positive findings (completely exclude?)
    excluded_labels = ['Edema','Hernia','Emphysema','Fibrosis','No Finding'
                      'Pleural_Thickening','Consolidation']
    """
    excluded_labels = ['No Finding']
    
    label_groups = details_df['Finding Labels'].unique()
    unique_labels = set([label for sublist in label_groups.tolist() for label in sublist.split('|')])
    
    # Drop some label that we do not want to include
    unique_labels = [l for l in unique_labels if l not in excluded_labels]

    index_to_label = {idx: val for idx, val in enumerate(unique_labels)}
    label_to_index = {val: idx for idx, val in index_to_label.items()}

    label_list = list(label_to_index.keys())

    return label_list, label_to_index

def create_image_list(dir):
    """ Create a full list of images available 
    Args:
        dir (string): root directory of images with subdirectories underneath
                      that have the .png images within them
    Returns:
        image_list: list of tuples with (image_name, full_image_path)
    """
    image_list = []
    dir = os.path.expanduser(dir)
    for subfolder in sorted(os.listdir(dir)):
        d = os.path.join(dir, subfolder)
        if not os.path.isdir(d):
            continue
        for subfolder_path, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(subfolder_path, fname)
                    image_list.append((fname, path))
    return image_list

def pil_loader(path):
    """ Opens path as file with Pillow (https://github.com/python-pillow/Pillow/issues/835)
    Args:
        path (string): File path to the image
    Returns:
        img: Image in RGB format
    """
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
        
def imshow(inp, title=None):
    """ Convert tensor array to an image (only use post-dataset transform) """
    inp = inp[0]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

### Implementation of Torch's Dataset

In [4]:
class XrayImageSet(Dataset):
    """
    Args:
        image_root (string): root directory of the images in form image/subfolder/*.png
        csv_file (string): path to the CSV data file
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
     Attributes:
        labels (list): list of the possible label names.
        label_to_index (dict): look from label name to a label index
        imgs (list): List of (filename, image path) tuples
    """
    
    def __init__(self, image_root, csv_file, transform=None, target_transform=None, loader = pil_loader):
        """ Create an instance of the Xray Dataset """
        img_details = pd.read_csv(csv_file)
        
        labels, label_to_index = create_label_maps(img_details)
        imgs = create_image_list(image_root)

        self.imgs = imgs
        self.image_details = img_details
        self.image_root = image_root
        self.labels = labels
        self.label_to_index = label_to_index
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.max_label_index = max(label_to_index.values())

    def __getitem__(self, index):
        """ Get image,labels pair by index
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        fname, path = self.imgs[index]
        target = self.get_one_hot_labels(fname)
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """ Calculate length of the dataset (number of images) """
        return len(self.imgs)
    
    def get_labels(self, fname):
        """ Return the label string for the file """
        return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]
    
    def one_hot_labels(self, labels):
        """ Convert the labels string (with each label separated by |) into 1-hot encoding """
        if labels == None:
            return None
        
        split_label_indices = [self.label_to_index.get(label)
                               for label in labels.split('|')
                               if label != 'No Finding']
        
        out = [1 if idx in split_label_indices else 0 for idx in range(self.max_label_index+1)]
        # This code UNHOTs the labels:
        # out = '|'.join([index_to_label.get(idx) for idx, val in enumerate(one_hot_tuple) if val == 1])
        return out

    def get_one_hot_labels(self, fname):
        """ Get the 1-hot encoded label array for the provided file """
        labels = self.get_labels(fname)
        one_hot_labels = self.one_hot_labels(labels)
        return torch.FloatTensor(one_hot_labels)

### Create the dataset with necessary transformations

In [5]:
img_transforms = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [6]:
img_data_train = XrayImageSet(image_root = '/user/images/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms,
                              target_transform = None)

img_data_train.imgs = [img for i, img in enumerate(img_data_train.imgs) if i % 10 > 0]# and randuni() < 0.1]

In [7]:
img_data_val   = XrayImageSet(image_root = '/user/images/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms,
                              target_transform = None)

img_data_val.imgs = [img for i, img in enumerate(img_data_val.imgs) if i % 10 == 0]# and randuni() < 0.1]

In [8]:
print("Training Set Size: {}".format(len(img_data_train)))
print("Validation Set Size: {}".format(len(img_data_val)))

Training Set Size: 100908
Validation Set Size: 11212


### Put the dataset into a Dataloader to handle batching

In [9]:
batch_size = 64
num_gpus = torch.cuda.device_count()
pin_mem_setting = True

print("Number of GPU: {}".format(num_gpus))

Number of GPU: 1


In [10]:
img_loader_train = DataLoader(img_data_train,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = 10,
                              pin_memory = pin_mem_setting)

img_loader_val   = DataLoader(img_data_val,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = 10,
                              pin_memory = pin_mem_setting)

In [11]:
dataloaders = {
    'train': img_loader_train,
    'val': img_loader_val
}

# Define model training procedure

In [12]:
class printer_writer:
    def __init__(self, output_folder_path):
        self.start_time = time.strftime('%Y%m%d-%Hh%Mm%Ss')
        
        self.outprefix = output_folder_path + '/' + self.start_time
        
        # Print Output File
        self.print_out_path = self.outprefix + '_print.txt'
        self.print_out_file = open(self.print_out_path, 'w', 1)
        
    def printw(self, string):
        print(string)
        try:
            self.print_out_file.write(string + "\n")
        except: # Ignore errors
            pass
        
    def save_checkpoint(self, epoch, model, optimizer, scheduler, val_error):
        model_out_path = self.outprefix + '_model_' + str(epoch+1) + '.tar'
        
        torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
        
    def close(self):
        self.print_out_file.close()

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, outfolder = '/user/xrayproj/output/'):
    since = time.time()
    scribe = printer_writer(outfolder)

    for epoch in range(num_epochs):
        scribe.printw('Epoch {}/{}'.format(epoch, num_epochs - 1))
        scribe.printw('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0
            obs_counter = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # Store statistics (convert from autograd.Variable to float/int)
                loss_val = loss.data[0]
                correct_val = torch.sum( ((outputs.sigmoid()>0.5) == (labels>0.5)).long() ).data[0]
                
                running_loss += loss_val
                running_corrects += correct_val
                
                obs_counter += len(inputs)
                
                batch_loss = 1.0 * loss_val / len(inputs)
                batch_acc = 1.0 * correct_val / len(inputs)
                status = ' |~~ {}@{}  Loss: {:.6f} Acc: {:.4f}'.format(
                    phase, obs_counter, batch_loss, batch_acc)
                scribe.printw(status)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            scribe.printw('{}  Loss: {:.6f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val':
                scribe.save_checkpoint(epoch, model, optimizer, scheduler, epoch_loss)

    time_elapsed = time.time() - since
    scribe.printw('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    scribe.close()

    return model

# Define Weighted Cost Metrics

In [14]:
def imbalance_weighted_bce_with_logit(input, target, size_average=True):
    if not (target.size() == input.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

    max_val = (-input).clamp(min=0)
    loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

    # Determine |P| and |N|
    positive_labels = target.sum()
    negative_labels = (1-target).sum()

    # Upweight the less common class (very often the 1s)
    beta_p = (positive_labels + negative_labels) / positive_labels
    beta_n = (positive_labels + negative_labels) / negative_labels

    # Adjust the losses accordingly
    loss_weight = target * beta_p + (1-target) * beta_n
    
    loss = loss * loss_weight

    if size_average:
        return loss.mean()
    else:
        return loss.sum()

In [15]:
class BCEWithLogitsImbalanceWeightedLoss(Module):
    def __init__(self, class_weight=None, size_average=True):
        super(BCEWithLogitsImbalanceWeightedLoss, self).__init__()
        self.size_average = size_average

    def forward(self, input, target):
        return imbalance_weighted_bce_with_logit(input, target, size_average=self.size_average)

# Setup Neural Network

### Define the model specifications

In [21]:
def ResNet18PlusFlexibleFC():
    # Create a base ResNet18 model
    m = models.resnet18(pretrained=True)
    for param in m.parameters():
        param.requires_grad = False

    # Replace the final FC layer
    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

In [42]:
def ResNet50PlusFlexibleFC():
    # Create a base ResNet18 model
    m = models.resnet50(pretrained=True)
    for param in m.parameters():
        param.requires_grad = False

    # Replace the final FC layer
    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

In [22]:
def ResNet18PlusFCFullyFlexible():
    # Create a base ResNet18 model
    m = models.resnet18(pretrained=True)

    # Replace the final FC layer
    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

### Pull the ResNet-18 pre-trained model and replace the fully connected layer at the end

In [48]:
#model_base = ResNet18PlusFlexibleFC()
#model_base = ResNet18PlusFCFullyFlexible()
model_base = ResNet50PlusFlexibleFC()

### Push model to CUDA/GPU

In [49]:
model_ft = DataParallel(model_base).cuda()

### Define loss measure and learning rates/procedures

In [50]:
#criterion = BCEWithLogitsImbalanceWeightedLoss()
criterion_base = nn.BCEWithLogitsLoss()

optimizer_ft = SGD(model_ft.module.fc.parameters(), lr=0.0001, momentum=0.9)
#optimizer_ft = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [51]:
criterion = criterion_base.cuda()

### Future code for allowing optimization of the base layer with a lower learning rate

```
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())

optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': opt.lr}
        ], lr=opt.lr*0.1, momentum=0.9)
```

# Begin Training Network (Normal Cost)

In [52]:
train_model(model_ft,
            criterion,
            optimizer_ft,
            exp_lr_scheduler,
            num_epochs=8)

Epoch 0/7
----------
 |~~ train@64  Loss: 0.010631 Acc: 7.9688
 |~~ train@128  Loss: 0.010612 Acc: 7.9219
 |~~ train@192  Loss: 0.010615 Acc: 8.0938
 |~~ train@256  Loss: 0.010579 Acc: 8.0938
 |~~ train@320  Loss: 0.010523 Acc: 8.2656
 |~~ train@384  Loss: 0.010541 Acc: 8.1250
 |~~ train@448  Loss: 0.010451 Acc: 8.4531
 |~~ train@512  Loss: 0.010437 Acc: 8.4375
 |~~ train@576  Loss: 0.010367 Acc: 8.7969
 |~~ train@640  Loss: 0.010313 Acc: 9.0156
 |~~ train@704  Loss: 0.010324 Acc: 8.7656
 |~~ train@768  Loss: 0.010167 Acc: 9.2344
 |~~ train@832  Loss: 0.010076 Acc: 9.4688
 |~~ train@896  Loss: 0.010046 Acc: 9.3906
 |~~ train@960  Loss: 0.009978 Acc: 9.7969
 |~~ train@1024  Loss: 0.009907 Acc: 9.8281
 |~~ train@1088  Loss: 0.009845 Acc: 10.0938
 |~~ train@1152  Loss: 0.009701 Acc: 10.3125
 |~~ train@1216  Loss: 0.009694 Acc: 10.5469
 |~~ train@1280  Loss: 0.009666 Acc: 10.4219
 |~~ train@1344  Loss: 0.009594 Acc: 10.5781
 |~~ train@1408  Loss: 0.009465 Acc: 10.9531
 |~~ train@1472  Loss

 |~~ train@11840  Loss: 0.004195 Acc: 13.3594
 |~~ train@11904  Loss: 0.004112 Acc: 13.3906
 |~~ train@11968  Loss: 0.004030 Acc: 13.4844
 |~~ train@12032  Loss: 0.004292 Acc: 13.2969
 |~~ train@12096  Loss: 0.004419 Acc: 13.1719
 |~~ train@12160  Loss: 0.004275 Acc: 13.2812
 |~~ train@12224  Loss: 0.004281 Acc: 13.2656
 |~~ train@12288  Loss: 0.003991 Acc: 13.4531
 |~~ train@12352  Loss: 0.004662 Acc: 13.0469
 |~~ train@12416  Loss: 0.004705 Acc: 13.0312
 |~~ train@12480  Loss: 0.004044 Acc: 13.3906
 |~~ train@12544  Loss: 0.004524 Acc: 13.1094
 |~~ train@12608  Loss: 0.004364 Acc: 13.1562
 |~~ train@12672  Loss: 0.004014 Acc: 13.4062
 |~~ train@12736  Loss: 0.004142 Acc: 13.3281
 |~~ train@12800  Loss: 0.003777 Acc: 13.5469
 |~~ train@12864  Loss: 0.004000 Acc: 13.4062
 |~~ train@12928  Loss: 0.003988 Acc: 13.4219
 |~~ train@12992  Loss: 0.004292 Acc: 13.2031
 |~~ train@13056  Loss: 0.004334 Acc: 13.1406
 |~~ train@13120  Loss: 0.004320 Acc: 13.2031
 |~~ train@13184  Loss: 0.004065 A

 |~~ train@23424  Loss: 0.003152 Acc: 13.4062
 |~~ train@23488  Loss: 0.003288 Acc: 13.3281
 |~~ train@23552  Loss: 0.003898 Acc: 13.0469
 |~~ train@23616  Loss: 0.003126 Acc: 13.3906
 |~~ train@23680  Loss: 0.003249 Acc: 13.3281
 |~~ train@23744  Loss: 0.003262 Acc: 13.3906
 |~~ train@23808  Loss: 0.003376 Acc: 13.2812
 |~~ train@23872  Loss: 0.003843 Acc: 13.0781
 |~~ train@23936  Loss: 0.003186 Acc: 13.3750
 |~~ train@24000  Loss: 0.003273 Acc: 13.3125
 |~~ train@24064  Loss: 0.003799 Acc: 13.1094
 |~~ train@24128  Loss: 0.003291 Acc: 13.3125
 |~~ train@24192  Loss: 0.003160 Acc: 13.4219
 |~~ train@24256  Loss: 0.003113 Acc: 13.4219
 |~~ train@24320  Loss: 0.003111 Acc: 13.4375
 |~~ train@24384  Loss: 0.003741 Acc: 13.1406
 |~~ train@24448  Loss: 0.004089 Acc: 12.9375
 |~~ train@24512  Loss: 0.003621 Acc: 13.1719
 |~~ train@24576  Loss: 0.003661 Acc: 13.1719
 |~~ train@24640  Loss: 0.003691 Acc: 13.1406
 |~~ train@24704  Loss: 0.003517 Acc: 13.1875
 |~~ train@24768  Loss: 0.003851 A

 |~~ train@34880  Loss: 0.002868 Acc: 13.3906
 |~~ train@34944  Loss: 0.003156 Acc: 13.2969
 |~~ train@35008  Loss: 0.002904 Acc: 13.3438
 |~~ train@35072  Loss: 0.003107 Acc: 13.2969
 |~~ train@35136  Loss: 0.003541 Acc: 13.1250
 |~~ train@35200  Loss: 0.003317 Acc: 13.1875
 |~~ train@35264  Loss: 0.003671 Acc: 13.0938
 |~~ train@35328  Loss: 0.003226 Acc: 13.2656
 |~~ train@35392  Loss: 0.003062 Acc: 13.2812
 |~~ train@35456  Loss: 0.002995 Acc: 13.3438
 |~~ train@35520  Loss: 0.002912 Acc: 13.3906
 |~~ train@35584  Loss: 0.003215 Acc: 13.2812
 |~~ train@35648  Loss: 0.003285 Acc: 13.2031
 |~~ train@35712  Loss: 0.003546 Acc: 13.1250
 |~~ train@35776  Loss: 0.003116 Acc: 13.3125
 |~~ train@35840  Loss: 0.002921 Acc: 13.3594
 |~~ train@35904  Loss: 0.003045 Acc: 13.2969
 |~~ train@35968  Loss: 0.003299 Acc: 13.2031
 |~~ train@36032  Loss: 0.003260 Acc: 13.1875
 |~~ train@36096  Loss: 0.003170 Acc: 13.2812
 |~~ train@36160  Loss: 0.003251 Acc: 13.2188
 |~~ train@36224  Loss: 0.003692 A

 |~~ train@46336  Loss: 0.003073 Acc: 13.2812
 |~~ train@46400  Loss: 0.002964 Acc: 13.2656
 |~~ train@46464  Loss: 0.003406 Acc: 13.1250
 |~~ train@46528  Loss: 0.003187 Acc: 13.2344
 |~~ train@46592  Loss: 0.002999 Acc: 13.2969
 |~~ train@46656  Loss: 0.003401 Acc: 13.1250
 |~~ train@46720  Loss: 0.002942 Acc: 13.2812
 |~~ train@46784  Loss: 0.002903 Acc: 13.2969
 |~~ train@46848  Loss: 0.003158 Acc: 13.2188
 |~~ train@46912  Loss: 0.002815 Acc: 13.3750
 |~~ train@46976  Loss: 0.003032 Acc: 13.3281
 |~~ train@47040  Loss: 0.003378 Acc: 13.1406
 |~~ train@47104  Loss: 0.003201 Acc: 13.2344
 |~~ train@47168  Loss: 0.002480 Acc: 13.4688
 |~~ train@47232  Loss: 0.002708 Acc: 13.3906
 |~~ train@47296  Loss: 0.002890 Acc: 13.3125
 |~~ train@47360  Loss: 0.002728 Acc: 13.4062
 |~~ train@47424  Loss: 0.002930 Acc: 13.2812
 |~~ train@47488  Loss: 0.002798 Acc: 13.3594
 |~~ train@47552  Loss: 0.002798 Acc: 13.3281
 |~~ train@47616  Loss: 0.002974 Acc: 13.2969
 |~~ train@47680  Loss: 0.002793 A

...

 |~~ train@61376  Loss: 0.002361 Acc: 13.4062
 |~~ train@61440  Loss: 0.002967 Acc: 13.2500
 |~~ train@61504  Loss: 0.003128 Acc: 13.2031
 |~~ train@61568  Loss: 0.002332 Acc: 13.4375
 |~~ train@61632  Loss: 0.002895 Acc: 13.2500
 |~~ train@61696  Loss: 0.003295 Acc: 13.2188
 |~~ train@61760  Loss: 0.002811 Acc: 13.3281
 |~~ train@61824  Loss: 0.003263 Acc: 13.1406
 |~~ train@61888  Loss: 0.003099 Acc: 13.2188
 |~~ train@61952  Loss: 0.003202 Acc: 13.1719
 |~~ train@62016  Loss: 0.002087 Acc: 13.5469
 |~~ train@62080  Loss: 0.002767 Acc: 13.2656
 |~~ train@62144  Loss: 0.003430 Acc: 13.0938
 |~~ train@62208  Loss: 0.002973 Acc: 13.2344
 |~~ train@62272  Loss: 0.002604 Acc: 13.3750
 |~~ train@62336  Loss: 0.002899 Acc: 13.2812
 |~~ train@62400  Loss: 0.003317 Acc: 13.1719
 |~~ train@62464  Loss: 0.002488 Acc: 13.3281
 |~~ train@62528  Loss: 0.002250 Acc: 13.4688
 |~~ train@62592  Loss: 0.003040 Acc: 13.2188
 |~~ train@62656  Loss: 0.002332 Acc: 13.4531
 |~~ train@62720  Loss: 0.002981 A

Process Process-166:
Process Process-164:
Process Process-165:
Process Process-162:
Process Process-161:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-167:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-169:
Traceback (most recent call last):
Process Process-170:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (

  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 783, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torchvision-0.1.9-py3.6.egg/torchvision/transforms/functional.py", line 197, in resize
    return img.resize((ow, oh), interpolation)
  File "<ipython-input-4-21cc0ac35a56>", line 57, in get_labels
    return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]
  File "<ipython-input-4-21cc0ac35a56>", line 42, in __getitem__
    target = self.get_one_hot_labels(fname)
  File "<ipython-input-4-21cc0ac35a56>", line 42, in __getitem__
    target = self.get_one_hot_labels(fname)
  File "<ipython-input-4-21cc0ac35a56>", line 42, in __getitem__
    target = self.get_one_hot_labels(fname)
  File "<ipython-input-3-4cc34eea5387>", line 68, in pil_loader
    return img.convert('RGB')
KeyboardInterrupt
  File "/home/ubuntu/anaconda3/lib/python3.6/site-pack

KeyboardInterrupt: 

  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/ImageFile.py", line 234, in load
    n, err_code = decoder.decode(b)
KeyboardInterrupt
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 783, in na_op
    result = _comp_method_OBJECT_ARRAY(op, x, y)
  File "<ipython-input-4-21cc0ac35a56>", line 42, in __getitem__
    target = self.get_one_hot_labels(fname)
  File "<ipython-input-4-21cc0ac35a56>", line 75, in get_one_hot_labels
    labels = self.get_labels(fname)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py", line 763, in _comp_method_OBJECT_ARRAY
    result = lib.scalar_compare(x, y, op)
KeyboardInterrupt
  File "<ipython-input-4-21cc0ac35a56>", line 57, in get_labels
    return self.image_details[self.image_details['Image Index'] =

### Save model results to S3

aws s3 cp ResNet18PlusFlexibleFC_Epoch9.tar s3://bdh-xrayproj-modelparameters/

import boto3

s3 = boto3.client('s3')
s3.list_buckets()

S3 Commands: http://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html

Boto3 QuickStart: http://boto3.readthedocs.io/en/latest/guide/quickstart.html

Key Management: https://aws.amazon.com/blogs/security/a-safer-way-to-distribute-aws-credentials-to-ec2/

AWS IAM Rules: http://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html

### Load model results back from S3

In [26]:
exp_lr_scheduler.last_epoch

9

# Analysis of Model Results

In [53]:
def compare_sums(model, dataset):
    model.train(False)
    
    obs_counter = 0
    total_pred = Variable(torch.FloatTensor(torch.zeros(14)), volatile=True)
    total_act = Variable(torch.FloatTensor(torch.zeros(14)), volatile=True)

    for data in dataset:
        inputs, labels = data
        
        inputs = Variable(inputs.cuda(), volatile=True)
        labels = Variable(labels.cuda(), volatile=True)

        outputs = model(inputs).sigmoid()

        total_act += labels.sum(0).cpu()
        total_pred += outputs.sum(0).cpu()

        obs_counter += len(inputs)
        
    return {'pred': total_pred / obs_counter,
            'act': total_act / obs_counter}

In [ ]:
compare_sums(model_ft, dataloaders['train'])

In [54]:
out_model_30.train(mode=False)

obs_counter = 0
total_pred = Variable(torch.FloatTensor(torch.zeros(14)))
total_act = Variable(torch.FloatTensor(torch.zeros(14)))

conf_a = {}
conf_b = {}
conf_c = {}
conf_d = {}
for i in range(1,10):
    conf_a[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_b[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_c[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_d[i] = Variable(torch.FloatTensor(torch.zeros(14)))

for data in dataloaders['val']:
    print("STARTING ITERATION...")
    inputs, labels = data
    print("PROCESSING FIRST {} OBSERVATIONS".format(len(inputs)))

    inputs = Variable(inputs.cuda())
    labels = Variable(labels.cuda())

    outputs = out_model_30(inputs).sigmoid()
    
    total_act += labels.sum(0).cpu()
    total_pred += outputs.sum(0).cpu()

    # Store statistics (convert from autograd.Variable to float/int)
    for i in range(1,10):
        t = i/10
        conf_a[i] += ((outputs.sigmoid()>t) == (labels>0.5)).sum(0).cpu().float()
        conf_b[i] += ((outputs.sigmoid()<t) == (labels>0.5)).sum(0).cpu().float()
        conf_c[i] += ((outputs.sigmoid()>t) == (labels<0.5)).sum(0).cpu().float()
        conf_d[i] += ((outputs.sigmoid()<t) == (labels<0.5)).sum(0).cpu().float()

    obs_counter += len(inputs)

STARTING ITERATION...
PROCESSING FIRST 750 OBSERVATIONS
STARTING ITERATION...
PROCESSING FIRST 406 OBSERVATIONS


In [67]:
comparison = Variable(torch.FloatTensor(9, 14))
for i in range(9):
    comparison[0] = conf_a[1] / obs_counter
print(comparison.int())

Variable containing:

Columns 0 to 5 
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -2.1475e+09  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00 -2.1475e+09  0.0000e+00 -2.1475e+09  0.0000e+00
 6.2634e+06 -1.0000e+00 -2.1475e+09  0.0000e+00 -2.1475e+09  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00

Columns 6 to 11 
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -6.5460e+04  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
-2.1475e

In [83]:
conf_d[9] / obs_counter

Variable containing:
 0.3183
 0.1462
 0.3356
 0.2171
 0.3166
 0.3183
 0.2803
 0.2898
 0.2232
 0.3279
 0.2846
 0.3045
 0.3002
 0.3192
[torch.FloatTensor of size 14]

In [84]:
img_data_train.labels

['Fibrosis',
 'Infiltration',
 'Hernia',
 'Effusion',
 'Emphysema',
 'Edema',
 'Cardiomegaly',
 'Mass',
 'Nodule',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Thickening',
 'Consolidation',
 'Pneumonia']

In [101]:
'''
torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
'''
test_load = torch.load('/user/xrayproj/output/20171120-01h41m56s_model_9.tar')

In [103]:
test_load.keys()

dict_keys(['epoch', 'state', 'optimizer', 'scheduler', 'val_error'])

In [106]:
load_opt = test_load['optimizer']
load_sched = test_load['scheduler']
load_state = test_load['state']

In [117]:
model2 = models.resnet18(pretrained=True)
for param in model2.parameters():
    param.requires_grad = False

# Replace FC layer
model2.fc = nn.Linear(model2.fc.in_features, len(img_data_train.labels))

model2_c = DataParallel(model2).cuda()

In [118]:
model2_c.load_state_dict(load_state)

In [123]:
model2_c.forward(Variable(img_data_train[0][0].unsqueeze(0).cuda())).sigmoid()

Variable containing:

Columns 0 to 9 
 0.7873  0.5093  0.2980  0.6386  0.4665  0.3371  0.1614  0.1846  0.1925  0.6630

Columns 10 to 13 
 0.0288  0.4468  0.3684  0.2549
[torch.cuda.FloatTensor of size 1x14 (GPU 0)]